# Explore here

In [224]:
# Your code here
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso

demographic_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")
demographic_data.to_csv("../data/raw/demographic_data.csv", index=False)
demographic_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [225]:
demographic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [226]:
demographic_data.isnull().sum()

fips                      0
TOT_POP                   0
0-9                       0
0-9 y/o % of total pop    0
19-Oct                    0
                         ..
CKD_prevalence            0
CKD_Lower 95% CI          0
CKD_Upper 95% CI          0
CKD_number                0
Urban_rural_code          0
Length: 108, dtype: int64

In [227]:
if demographic_data.duplicated().sum():
    demographic_data = demographic_data.drop_duplicates()

In [228]:
demographic_data.describe()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
count,3140.000000,3.140000e+03,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,...,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000
mean,30401.640764,1.041894e+05,1.274030e+04,11.871051,1.336798e+04,12.694609,1.446933e+04,12.283979,1.391649e+04,11.751535,...,5827.242357,13.073503,12.088089,14.053726,9326.577707,3.446242,3.207516,3.710478,2466.234076,4.635350
std,15150.559265,3.335834e+05,4.180730e+04,2.124081,4.228439e+04,1.815044,4.957773e+04,3.126297,4.899095e+04,1.696599,...,15720.551934,2.724351,2.622948,2.824828,29754.601185,0.568059,0.527740,0.613069,7730.422067,1.510447
min,1001.000000,8.800000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.100000e+01,6.092789,...,7.000000,6.100000,5.500000,6.700000,11.000000,1.800000,1.700000,1.900000,3.000000,1.000000
25%,18180.500000,1.096325e+04,1.280500e+03,10.594639,1.374500e+03,11.674504,1.263750e+03,10.496774,1.232750e+03,10.689322,...,815.000000,11.200000,10.300000,12.100000,1187.750000,3.100000,2.900000,3.300000,314.750000,3.000000
50%,29178.000000,2.580050e+04,3.057000e+03,11.802727,3.274000e+03,12.687422,3.108000e+03,11.772649,3.000500e+03,11.580861,...,1963.500000,12.800000,11.800000,13.800000,2743.000000,3.400000,3.200000,3.700000,718.000000,5.000000
75%,45081.500000,6.791300e+04,8.097000e+03,12.951840,8.822250e+03,13.659282,8.976250e+03,13.182260,8.314250e+03,12.639379,...,4727.000000,14.800000,13.700000,15.900000,6679.250000,3.800000,3.500000,4.100000,1776.250000,6.000000
max,56045.000000,1.010552e+07,1.208253e+06,25.460677,1.239139e+06,23.304372,1.557073e+06,37.570198,1.501844e+06,22.225129,...,434075.000000,25.600000,24.200000,27.000000,952335.000000,6.200000,5.800000,6.600000,237766.000000,6.000000


In [229]:
object_columns = demographic_data.select_dtypes(include='object').columns

for col in object_columns:
    demographic_data[col], _ = pd.factorize(demographic_data[col])


In [230]:
scaler = MinMaxScaler()

numeric_columns = demographic_data.select_dtypes(include=["float64", "int64"]).columns
scaler_features = scaler.fit_transform(demographic_data[numeric_columns])
demographic_data_scal = pd.DataFrame(scaler_features, index=demographic_data.index, columns=numeric_columns)

demographic_data_scal.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,0.000000,0.005493,0.005617,0.479430,0.006163,0.589390,0.004417,0.329258,0.004713,0.412648,...,0.008379,0.348718,0.342246,0.349754,0.005724,0.295455,0.292683,0.297872,0.005564,0.4
1,0.000036,0.021566,0.020490,0.445993,0.021719,0.529693,0.015143,0.287860,0.016781,0.339172,...,0.033831,0.302564,0.294118,0.315271,0.021536,0.318182,0.317073,0.340426,0.023031,0.6
2,0.000073,0.002453,0.002261,0.431264,0.002389,0.510489,0.002099,0.349599,0.002124,0.419806,...,0.005451,0.697436,0.700535,0.684729,0.004052,0.613636,0.609756,0.617021,0.003718,1.0
3,0.000109,0.002208,0.002033,0.430636,0.002095,0.497301,0.001945,0.359921,0.002065,0.483782,...,0.004105,0.410256,0.411765,0.403941,0.002625,0.340909,0.341463,0.361702,0.002490,0.2
4,0.000145,0.005715,0.005872,0.481786,0.006109,0.561604,0.004330,0.310254,0.004576,0.360085,...,0.010722,0.379487,0.379679,0.384236,0.006307,0.363636,0.365854,0.382979,0.006326,0.2


In [231]:
X = demographic_data_scal.drop(columns=["Heart disease_number"])
y = demographic_data_scal["Heart disease_number"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [232]:
# selection_model = SelectKBest(f_regression, k=15)
# selection_model.fit(X_train, y_train)

# selected_columns = X_train.columns[selection_model.get_support()]

# X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns= selected_columns)
# X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns= selected_columns)

# X_train_sel.head()
train_indices = list(X_train.index)
test_indices = list(X_test.index)

k = int(len(X_train.columns) * 0.3)
selection_model = SelectKBest(f_regression, k = k)
selection_model.fit(X_train, y_train)
index = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = X_train.columns.values[index])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = X_test.columns.values[index])

X_train_sel.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number
0,0.002626,0.002666,0.002795,0.001877,0.001876,0.002094,0.002923,0.003786,0.004286,0.003391,...,0.004124,0.003594,0.002597,0.003994,0.002584,0.003535,0.003768,0.005315,0.002953,0.003230
1,0.005064,0.004363,0.004641,0.003299,0.003549,0.004352,0.006027,0.008135,0.008375,0.008218,...,0.007420,0.007129,0.005225,0.008457,0.005206,0.006687,0.006525,0.009422,0.005676,0.006103
2,0.003730,0.003240,0.004130,0.003983,0.002898,0.003294,0.003900,0.004268,0.004728,0.004381,...,0.004824,0.003266,0.003708,0.004461,0.003826,0.004950,0.004868,0.006416,0.003872,0.003651
3,0.009094,0.009239,0.010205,0.007447,0.007555,0.008474,0.009770,0.010619,0.011755,0.009762,...,0.013571,0.007236,0.008908,0.010805,0.008895,0.011475,0.012255,0.013152,0.008298,0.008895
4,0.013300,0.013820,0.014257,0.011098,0.010642,0.011733,0.013758,0.016651,0.017585,0.016950,...,0.020864,0.018186,0.013285,0.017330,0.013086,0.017278,0.018522,0.023026,0.013626,0.014666


In [233]:
X_test_sel.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number
0,0.000886,0.000672,0.000729,0.000711,0.000826,0.000896,0.000959,0.001079,0.001378,0.001620,...,0.001011,0.000823,0.000871,0.001318,0.000934,0.001196,0.001308,0.001675,0.001151,0.001220
1,0.026690,0.025527,0.024168,0.026632,0.024056,0.022145,0.026777,0.034480,0.037981,0.027445,...,0.040725,0.026504,0.025612,0.032998,0.027091,0.029710,0.030363,0.029479,0.022856,0.025122
2,0.001714,0.001718,0.001718,0.001441,0.001603,0.001792,0.001779,0.001865,0.002332,0.001476,...,0.001899,0.001668,0.001749,0.002012,0.001710,0.003022,0.002813,0.005257,0.002723,0.002486
3,0.011602,0.012563,0.012825,0.008699,0.009365,0.010526,0.012865,0.014825,0.014752,0.011497,...,0.018201,0.015865,0.011288,0.013714,0.011225,0.012801,0.012963,0.018357,0.009888,0.011087
4,0.022324,0.020274,0.021830,0.026017,0.018913,0.018823,0.020179,0.027425,0.030236,0.023224,...,0.034064,0.022168,0.021044,0.026288,0.022950,0.020906,0.021737,0.025346,0.017799,0.021244


In [234]:
X_train_sel["Heart disease_number"] = list(y_train)
X_test_sel["Heart disease_number"] = list(y_test)

X_train_sel.to_csv("../data/processed/clean_train.csv", index = False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index = False)

In [235]:
train_data = pd.read_csv("../data/processed/clean_train.csv")
test_data = pd.read_csv("../data/processed/clean_test.csv")

train_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number,Heart disease_number
0,0.002626,0.002666,0.002795,0.001877,0.001876,0.002094,0.002923,0.003786,0.004286,0.003391,...,0.003594,0.002597,0.003994,0.002584,0.003535,0.003768,0.005315,0.002953,0.003230,0.004749
1,0.005064,0.004363,0.004641,0.003299,0.003549,0.004352,0.006027,0.008135,0.008375,0.008218,...,0.007129,0.005225,0.008457,0.005206,0.006687,0.006525,0.009422,0.005676,0.006103,0.008715
2,0.003730,0.003240,0.004130,0.003983,0.002898,0.003294,0.003900,0.004268,0.004728,0.004381,...,0.003266,0.003708,0.004461,0.003826,0.004950,0.004868,0.006416,0.003872,0.003651,0.005094
3,0.009094,0.009239,0.010205,0.007447,0.007555,0.008474,0.009770,0.010619,0.011755,0.009762,...,0.007236,0.008908,0.010805,0.008895,0.011475,0.012255,0.013152,0.008298,0.008895,0.012597
4,0.013300,0.013820,0.014257,0.011098,0.010642,0.011733,0.013758,0.016651,0.017585,0.016950,...,0.018186,0.013285,0.017330,0.013086,0.017278,0.018522,0.023026,0.013626,0.014666,0.019962


In [236]:
test_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number,Heart disease_number
0,0.000886,0.000672,0.000729,0.000711,0.000826,0.000896,0.000959,0.001079,0.001378,0.001620,...,0.000823,0.000871,0.001318,0.000934,0.001196,0.001308,0.001675,0.001151,0.001220,0.001589
1,0.026690,0.025527,0.024168,0.026632,0.024056,0.022145,0.026777,0.034480,0.037981,0.027445,...,0.026504,0.025612,0.032998,0.027091,0.029710,0.030363,0.029479,0.022856,0.025122,0.032142
2,0.001714,0.001718,0.001718,0.001441,0.001603,0.001792,0.001779,0.001865,0.002332,0.001476,...,0.001668,0.001749,0.002012,0.001710,0.003022,0.002813,0.005257,0.002723,0.002486,0.004050
3,0.011602,0.012563,0.012825,0.008699,0.009365,0.010526,0.012865,0.014825,0.014752,0.011497,...,0.015865,0.011288,0.013714,0.011225,0.012801,0.012963,0.018357,0.009888,0.011087,0.015484
4,0.022324,0.020274,0.021830,0.026017,0.018913,0.018823,0.020179,0.027425,0.030236,0.023224,...,0.022168,0.021044,0.026288,0.022950,0.020906,0.021737,0.025346,0.017799,0.021244,0.025985


In [237]:
X_train = train_data.drop(["Heart disease_number"], axis = 1)
y_train = train_data["Heart disease_number"]
X_test = test_data.drop(["Heart disease_number"], axis = 1)
y_test = test_data["Heart disease_number"]

In [238]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [239]:
print(f"Intercepto (a): {model.intercept_}")
print(f"Coeficientes (b): {model.coef_}")

Intercepto (a): -1608396.229716847
Coeficientes (b): [-5.53722503e+11  5.88893809e+10  6.03947423e+10  7.58906164e+10
  7.31982586e+10  6.58015368e+10  6.36529631e+10  4.94364951e+10
  2.79044074e+10  1.73645869e+10  1.24524810e-01  6.11911239e+10
  1.15893708e-01  9.31012188e-03  1.25928920e-01  1.15988499e-01
  2.26472496e-02  2.80914307e-02 -9.97352600e-02 -3.12232971e-02
  1.28440857e-02 -3.60832214e-02 -5.39093018e-02  4.70352173e-03
 -1.21675873e+00  5.66730499e-01  3.07471085e+00 -3.65715027e-02
  1.42070770e-01  3.24489594e-01  1.46034241e-01  2.33985901e-01]


In [240]:
y_pred = model.predict(X_test)
y_pred

array([1.62473042e-03, 3.01996842e-02, 3.86028178e-03, 1.58746678e-02,
       2.57602064e-02, 3.16497241e-03, 1.70442928e-03, 3.00374161e-03,
       8.89012357e-04, 1.55281741e-03, 2.98317522e-03, 5.12067461e-04,
       9.84412059e-03, 3.22166830e-03, 2.12021845e-02, 5.70678571e-03,
       1.23691642e-01, 2.77897646e-03, 4.45204158e-03, 9.97547992e-04,
       3.48309544e-03, 1.58201437e-03, 1.93833187e-03, 1.34320999e-03,
       2.25527791e-01, 5.56231174e-03, 2.00274144e-03, 4.90769697e-03,
       1.97931728e-03, 4.18314710e-04, 2.65465071e-03, 4.23172815e-03,
       6.03174046e-03, 7.39474525e-03, 2.00174898e-02, 3.51402070e-03,
       6.06948580e-03, 2.80857831e-03, 1.97013537e-03, 4.09610872e-03,
       3.13508790e-03, 2.07999405e-02, 8.39286949e-04, 3.44160013e-04,
       5.95887774e-03, 1.66401244e-03, 7.82986870e-04, 1.78244491e-02,
       2.47452362e-03, 1.73131726e-03, 4.21731137e-02, 7.22462311e-04,
       3.27995303e-03, 1.27203204e-03, 1.95916311e-02, 1.68951508e-03,
      

In [241]:
print(f"Error cuadrático medio: {mean_squared_error(y_test, y_pred)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 1.040827556365026e-06
Coeficiente de determinación: 0.9977520357236916


In [242]:
lasso_model = Lasso(alpha = 0.01)
lasso_model.fit(X_train, y_train)

score = lasso_model.score(X_test, y_test)
print("Coefficients:", lasso_model.coef_)
print("R2 score:", score)


Coefficients: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
R2 score: -0.016919136501585053


In [243]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=0.01, max_iter=300)
ridge_model.fit(X_train, y_train)
score = ridge_model.score(X_test, y_test)

print("Coefficients:", ridge_model.coef_)
print("R2 score:", score)

Coefficients: [-0.00117392  0.0422655  -0.04570643 -0.0779873  -0.03443099  0.01912642
 -0.00222414  0.01863675  0.1702699   0.07730971  0.06433984 -0.00117392
  0.09308336 -0.01338901  0.04956625  0.11465078  0.00116146  0.00639135
 -0.10561547  0.01575095  0.02886737 -0.06905985 -0.04169588  0.00050105
 -0.05988801  0.05619588 -0.00231051  0.08505852  0.12483998  0.28987062
  0.08934865  0.12307179]
R2 score: 0.9975421877565914
